In [1]:
a = """
.model jjmod jj(Rtype=1, Vg=2.8mV, Cap=0.064pF, R0=100ohm, Rn=16ohm, Icrit=0.1mA)
.model pjjmod jj(Rtype=1, Vg=2.8mV, Cap=0.064pF, R0=100ohm, Rn=16ohm, Icrit=0.1mA, PHI=PI)

*PYSQUID=========================================================================

.subckt psquid        3  5 
L1                 3         1   1.75pH fcheck
L2                 3         4   1.75pH fcheck
B1                 1         5  jjmod area=0.49
R1                 1         5   15.88ohm
B2                 2         5  jjmod area=0.49
R2                 2         5   15.88ohm
B3                 4         2  pjjmod area=20
R3                 4         2   0.2ohm
.ends

.subckt psquid1        3  5 
L1                 3         1   #@SQUIDL1(1.75)pH fcheck
L2                 3         4   #@SQUIDL1(1.75)pH fcheck
B1                 1         5  jjmod area=#JJAREA1(0.49)
R1                 1         5   #@JJSHUNT1(16)ohm
B2                 2         5  jjmod area=#JJAREA1(0.49)
R2                 2         5   #@JJSHUNT1(16)ohm
B3                 4         2  pjjmod area=#PJJAREA1(20)
R3                 4         2   #@PJJSHUNT1(0.2)ohm
.ends

.subckt psquid2        3  5 
L1                 3         1   #@SQUIDL2(1.75)pH fcheck
L2                 3         4   #@SQUIDL2(1.75)pH fcheck
B1                 1         5  jjmod area=#JJAREA2(v=0.49,fixed=true,)
R1                 1         5   #@JJSHUNT2(16)ohm
B2                 2         5  jjmod area=#JJAREA2(0.49)
R2                 2         5   #@JJSHUNT2(16)ohm
B3                 4         2  pjjmod area=#PJJAREA2(20)
R3                 4         2   #@PJJSHUNT2(0.2)ohm
.ends

.subckt psquid3        3  5 
L1                 3         1   #@SQUIDL3(1.75)pH fcheck
L2                 3         4   #@SQUIDL3(1.75)pH fcheck
B1                 1         5  jjmod area=#JJAREA3(0.49)
R1                 1         5   #@JJSHUNT3(16)ohm
B2                 2         5  jjmod area=#JJAREA3(0.49)
R2                 2         5   #@JJSHUNT3(16)ohm
B3                 4         2  pjjmod area=#PJJAREA3(20)
R3                 4         2   #@PJJSHUNT3(0.2)ohm
.ends

*================================================================================



.subckt JTL     1  5  100
L1               1  2   18pH fcheck
L2               2  3   18pH fcheck
L3               3  4   #JTLL1(18)pH fcheck
L4               4  5   #JTLL1(18)pH fcheck
X1    psquid     3  0
X2    psquid     5  0
R1               100  4   #JTLR1(100)ohm
.ends



.subckt JTL4    1  5  100
X1       JTL            1  2  100
X2       JTL            2  3  100
X3       JTL            3  4  100
X4       JTL            4  5  100
.ends


.subckt DFF 1 4 9 100
L1                   1  2  #DFFL1(10)pH fcheck  
X1       psquid1      2  3
L2                   3  7  #DFFL2(18)pH fcheck

L3                   4  5  #DFFL3(18)pH fcheck
L4                   5  6  #DFFL4(18)pH fcheck
X2       psquid2      6  0
L5                   6  7  2pH  fcheck
R1                   100  7  #DFFR1(100)ohm
L6                   7  8  #DFFL6(10)pH fcheck
L7                   8  9  2pH fcheck
X3       psquid3      9  0
.ends


*** top cell: 
Vin1                     2    0      PWL(0ps 0uV  400ps 0mV 401ps 0.517mV 402ps 0.517mV 403ps 0mV  600ps 0mV 601ps 0.517mV 602ps 0.517mV 603ps 0mV  800ps 0mV 801ps 0.517mV 802ps 0.517mV 803ps 0mV)
X2       JTL4          2     3  100
X3       JTL4          3     4  100
Vin2                     6    0      PWL(0ps 0uV  300ps 0mV 301ps 0.517mV 302ps 0.517mV 303ps 0mV  500ps 0mV 501ps 0.517mV 502ps 0.517mV 503ps 0mV  700ps 0mV 701ps 0.517mV 702ps 0.517mV 703ps 0mV   900ps 0mV 901ps 0.517mV 902ps 0.517mV 903ps 0mV)
X4       JTL4          6     7  100
X5       JTL4          7     8  100
X6       DFF           4     8  9 100
X7       JTL4          9     10  100
X8       JTL4          10    11  100


X9       psquid        11     0
R1                     11    12  8.32ohm
L1                     12     0  2pH fcheck
Vb                     100    0  pwl(0ps 0mV 100ps 1.2mV)



**netlis file
.tran 1.0ps 6000ps 0ps 1ps

.print phase B1|X2|X6
.print phase B2|X2|X6

.print phase B1|X1|X1|X8
.print phase B2|X1|X1|X8

.end

**** optimize ****
* バイアス電圧の立ち上がり終了の時間
* EndTimeOfBiasRise=100e-12
* 初期パルスの入力開始時間
* StartTimeOfPulseInput=300e-12
"""

In [2]:
def isint(s):  # 正規表現を使って判定を行う
    p = '[-+]?\d+'
    return True if re.fullmatch(p, s) else False

def isfloat(s):  # 正規表現を使って判定を行う
    p = '[-+]?(\d+\.?\d*|\.\d+)([eE][-+]?\d+)?'
    return True if re.fullmatch(p, s) else False

def digit(s):  # 正規表現を使って小数点以下の桁数
    if re.search("\.",s)!= None:
        return len(re.split("\.",s)[1])
    else:
        return 0

def stringToNum(s):
    if isint(s):
        return int(s)

    elif isfloat(s):
        return float(s)

    else:
        raise ValueError("値が数値ではありません。")

In [3]:
import re
import pandas as pd


  


In [6]:
df = pd.DataFrame(columns=['char','text','default','value','fix','lic','bc','global'])  
df.set_index('char', inplace=True)
vlist = re.findall('#.+\(.+\)',a)
for raw_line in vlist:
    li = re.sub('\s','',raw_line)
    char = re.search('#.+\(',li).group()
    char = re.sub('#|\(','',char)
    if char in df.index:
        continue
    
    dic = {'text': None,'default': None,'value': None,'fix': False,'lic': None,'bc': None,'global': None}
    dic['text']='#'+char
    m = re.search('\(.+\)',li).group()
    m = re.sub('\(|\)','',m)
    spl = re.split(',',m)
    if len(spl)==1:
        if isfloat(spl[0]) or isint(spl[0]):
            num = stringToNum(spl[0])
            dic['default'] = num
            dic['value'] = num
    for sp in spl:
        val = re.split('=',sp)
        if len(val) == 1:
            if isfloat(val[0]) or isint(val[0]):
                num = stringToNum(spl[0])
                dic['default'] = num
                dic['value'] = num
        elif len(val) == 2:
            if re.fullmatch('v|value',val[0],flags=re.IGNORECASE):
                num = stringToNum(val[1])
                dic['default'] = num
                dic['value'] = num
            elif re.fullmatch('fix|fixed',val[0],flags=re.IGNORECASE):
                if re.fullmatch('true',val[1],flags=re.IGNORECASE):
                    dic['fix'] = True
                else:
                    dic['fix'] = False
            elif re.fullmatch('lic',val[0],flags=re.IGNORECASE):
                if re.fullmatch('(L|Ic|I)\d+',val[1],flags=re.IGNORECASE):
                    dic['lic'] = val[1]
                else:
                    raise ValueError("[ "+sp+" ]のLIc積の記述が読み取れません。")
            elif re.fullmatch('(β|b)c',val[0],flags=re.IGNORECASE):
                if re.fullmatch('(A|R)\d+',val[1],flags=re.IGNORECASE):
                    dic['bc'] = val[1]
                else:
                    raise ValueError("[ "+sp+" ]のbcの記述が読み取れません。")
            elif re.fullmatch('global',val[0],flags=re.IGNORECASE):
                num = stringToNum(val[1])
                dic['global'] = num
            else:
                raise ValueError("[ "+sp+" ]の記述が読み取れません。")
        else:
            raise ValueError("[ "+sp+" ]の記述が読み取れません。")
    
    if dic['global'] == None:
        for line in a.splitlines():
            if raw_line in line:
                if re.fullmatch('R',line[0:1],flags=re.IGNORECASE):
                    dic['global'] = 10
                elif re.fullmatch('L',line[0:1],flags=re.IGNORECASE):
                    dic['global'] = 10
                elif re.fullmatch('C',line[0:1],flags=re.IGNORECASE):
                    dic['global'] = 5
                elif re.fullmatch('V',line[0:1],flags=re.IGNORECASE):
                    dic['global'] = 5
                elif re.fullmatch('B',line[0:1],flags=re.IGNORECASE):
                    dic['global'] = 5
                break
    
    df.loc[char] = dic
    
    print(dic)
 
print(df)

{'text': '#@SQUIDL1', 'default': 1.75, 'value': 1.75, 'fix': False, 'lic': None, 'bc': None, 'global': 10}
{'text': '#JJAREA1', 'default': 0.49, 'value': 0.49, 'fix': True, 'lic': 'L1', 'bc': 'A2', 'global': 43}
{'text': '#@JJSHUNT1', 'default': 16, 'value': 16, 'fix': False, 'lic': None, 'bc': None, 'global': 10}
{'text': '#PJJAREA1', 'default': 20, 'value': 20, 'fix': False, 'lic': None, 'bc': None, 'global': 5}
                 text  default  value    fix   lic    bc global
char                                                           
@SQUIDL1    #@SQUIDL1     1.75   1.75  False  None  None     10
JJAREA1      #JJAREA1     0.49   0.49   True    L1    A2     43
@JJSHUNT1  #@JJSHUNT1    16.00  16.00  False  None  None     10
PJJAREA1    #PJJAREA1    20.00  20.00  False  None  None      5


In [16]:
print(a)

for m in re.findall('#.+\(.+\)',a):
    print(m)
    char = re.search('#.+\(',m).group()
    char = re.sub('#|\(','',char)
    char = "#("+char+")"
    a = a.replace(m,)

print(a)


.subckt psquid1        3  5 
L1                 3         1   #@SQUIDL1pH fcheck
L2                 3         4   #@SQUIDL1pH fcheck
B1                 1         5  jjmod area=#JJAREA1
R1                 1         5   #@JJSHUNT1ohm
B2                 2         5  jjmod area=#JJAREA1
R2                 2         5   #@JJSHUNT1ohm
B3                 4         2  pjjmod area=#PJJAREA1


.subckt psquid1        3  5 
L1                 3         1   #@SQUIDL1pH fcheck
L2                 3         4   #@SQUIDL1pH fcheck
B1                 1         5  jjmod area=#JJAREA1
R1                 1         5   #@JJSHUNT1ohm
B2                 2         5  jjmod area=#JJAREA1
R2                 2         5   #@JJSHUNT1ohm
B3                 4         2  pjjmod area=#PJJAREA1

